# Red Neuronal (?) Inception V3 con Tensor.flow Keras

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os
import keras
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input
import pandas as pd
import numpy as np



In [2]:
cumulos_folder_shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell1\cumulos"
nocumulos_folder_shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell1\no-cumulos"

cumulos_folder_shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell2\cumulos"
nocumulos_folder_shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell2\no-cumulos"

shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell1"
shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell2"

clasificar_shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2_diagramas\shell1P"
clasificar_shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2_diagramas\shell2"

In [53]:
# Paths to the datasets
train_data_dir = shell1 # Cambiar a shell1 o shell2 segun corresponda

#train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescale pixel values to [0, 1]
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False
)

# Load training data from 'clusters' and 'not_clusters'
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="rgb",
    target_size=(253, 385),  # InceptionV3 requires 299x299 input size
    batch_size=32,
    class_mode='binary',  # Binary classification: "clusters" vs "not_clusters"
    classes=['cumulos', 'no-cumulos'],  # Map directories to labels
    shuffle=True
)


Found 10272 images belonging to 2 classes.


In [ ]:

# Data preprocessing for the unknown folder (without augmentation)
#test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_data_dir=clasificar_shell1

#test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescale pixel values to [0, 1]
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    #dtype=tf.float32
)

unknown_generator = test_datagen.flow_from_directory(
    test_data_dir, 
    target_size=(253, 385),
    batch_size=1,
    color_mode="rgb",
    class_mode=None,  # No labels for unknown data
    #shuffle=False  # Keep the order of files
)


Found 37 images belonging to 1 classes.


In [16]:
def get_model():
  base_model = InceptionV3(input_shape = (253,385,3), weights='imagenet', include_top=False)
  x = base_model.output
  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dense(1024, activation='relu')(x)
  x = keras.layers.BatchNormalization()(x)
  predictions = keras.layers.Dense(1, activation='sigmoid')(x)
  model = keras.models.Model(inputs=base_model.input, outputs=predictions)

  for layer in base_model.layers:
    layer.trainable = False
  
  model.summary()
  return model

# Compile the model

In [17]:
model = get_model()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 253, 385,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_282 (Conv2D) │ (None, 126, 192,  │        864 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 126, 192,  │         96 │ conv2d_282[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_282      │ (None, 126, 192,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_283 (Conv2D) │ (None, 124, 190,  │      9,216 │ activation_282[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 124, 190,  │         96 │ conv2d_283[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_283      │ (None, 124, 190,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_284 (Conv2D) │ (None, 124, 190,  │     18,432 │ activation_283[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 124, 190,  │        192 │ conv2d_284[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_284      │ (None, 124, 190,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 61, 94,    │          0 │ activation_284[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_285 (Conv2D) │ (None, 61, 94,    │      5,120 │ max_pooling2d_12… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 94,    │        240 │ conv2d_285[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_285      │ (None, 61, 94,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_286 (Conv2D) │ (None, 59, 92,    │    138,240 │ activation_285[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 59, 92,    │        576 │ conv2d_286[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_286      │ (None, 59, 92,    │          0 │ batch_normalizat

 Total params: 23,906,081 (91.19 MB)

 Trainable params: 2,101,249 (8.02 MB)

 Non-trainable params: 21,804,832 (83.18 MB)

In [ ]:
# Compile the model
#model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',    metrics=['accuracy'])
model.compile(keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])


321/321 ━━━━━━━━━━━━━━━━━━━━ 676s 2s/step - accuracy: 0.8885 - loss: 0.3503


In [ ]:

model.fit(train_generator,epochs=1)
#model.fit(
#    train_generator,
#    epochs=2,  # Adjust based on your dataset
    #steps_per_epoch=train_generator.samples // train_generator.batch_size
#)

In [51]:
#img_path =  r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2_diagramas\shell1P\dmwidmw\Alessi_91_3.png"
#img_path = r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2_diagramas\shell1\prueba\Q8_cluster_100.csv.png"
resultados={}
folder_path = r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2_diagramas\shell1\prueba"
for file in os.listdir(folder_path):
    img_path=os.path.join(folder_path,file)
    img = image.load_img(img_path, target_size=(349, 291))
 # Preprocessing the image
    x = image.img_to_array(img)
# x = np.true_divide(x, 255)
## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
#preds=np.argmax(preds, axis=1)

    print(file+"   :"+str(preds))
    resultados[file]=preds[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
Q1_cluster_0.png   :[[0.6872629]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Q1_cluster_1.csv.png   :[[0.14621611]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Q1_cluster_10.csv.png   :[[0.03024296]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Q1_cluster_11.csv.png   :[[0.29701477]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Q1_cluster_12.csv.png   :[[0.04924019]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Q1_cluster_13.csv.png   :[[0.03683276]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Q1_cluster_14.csv.png   :[[0.4568245]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Q1_cluster_15.csv.png   :[[0.14583468]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Q1_cluster_16.csv.png   :[[0.08092151]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Q1_cluster_17.csv.png   :[[0.94305575]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Q1_cluster_18.csv.png   :[[0.9103956]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Q1_cluster_19.csv.png   :[[0.78772974]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Q1_cluster_2.csv.png  

In [52]:
resultados

{'Q1_cluster_0.png': 0.6872629,
 'Q1_cluster_1.csv.png': 0.14621611,
 'Q1_cluster_10.csv.png': 0.030242965,
 'Q1_cluster_11.csv.png': 0.29701477,
 'Q1_cluster_12.csv.png': 0.04924019,
 'Q1_cluster_13.csv.png': 0.036832765,
 'Q1_cluster_14.csv.png': 0.4568245,
 'Q1_cluster_15.csv.png': 0.14583468,
 'Q1_cluster_16.csv.png': 0.08092151,
 'Q1_cluster_17.csv.png': 0.94305575,
 'Q1_cluster_18.csv.png': 0.9103956,
 'Q1_cluster_19.csv.png': 0.78772974,
 'Q1_cluster_2.csv.png': 0.060793888,
 'Q1_cluster_20.csv.png': 0.9532666,
 'Q1_cluster_21.csv.png': 0.9050151,
 'Q1_cluster_22.csv.png': 0.98146135,
 'Q1_cluster_23.csv.png': 0.5473919,
 'Q1_cluster_24.csv.png': 0.9802056,
 'Q1_cluster_25.csv.png': 0.99189234,
 'Q1_cluster_26.csv.png': 0.9697321,
 'Q1_cluster_27.csv.png': 0.997905,
 'Q1_cluster_28.csv.png': 0.9988547,
 'Q1_cluster_29.csv.png': 0.9666632,
 'Q1_cluster_3.csv.png': 0.99012154,
 'Q1_cluster_30.csv.png': 0.9965575,
 'Q1_cluster_31.csv.png': 0.993833,
 'Q1_cluster_32.csv.png': 0.9554

In [54]:
filtro_resultados={}
llaves=resultados.keys()
for x in llaves:
    valor=resultados[x]
    if valor <0.5:
        filtro_resultados[x]=valor
        

In [57]:
filtro_resultados

{'Q1_cluster_1.csv.png': 0.14621611,
 'Q1_cluster_10.csv.png': 0.030242965,
 'Q1_cluster_11.csv.png': 0.29701477,
 'Q1_cluster_12.csv.png': 0.04924019,
 'Q1_cluster_13.csv.png': 0.036832765,
 'Q1_cluster_14.csv.png': 0.4568245,
 'Q1_cluster_15.csv.png': 0.14583468,
 'Q1_cluster_16.csv.png': 0.08092151,
 'Q1_cluster_2.csv.png': 0.060793888,
 'Q1_cluster_35.csv.png': 0.49647808,
 'Q1_cluster_4.csv.png': 0.0787797,
 'Q1_cluster_5.csv.png': 0.0654277,
 'Q1_cluster_6.csv.png': 0.2550093,
 'Q1_cluster_66.csv.png': 0.43436489,
 'Q1_cluster_79.csv.png': 0.23192808,
 'Q1_cluster_8.csv.png': 0.39009085,
 'Q1_cluster_9.csv.png': 0.06811509,
 'Q2_cluster_2.csv.png': 0.30605793,
 'Q2_cluster_23.csv.png': 0.19664036,
 'Q2_cluster_28.csv.png': 0.39551684,
 'Q2_cluster_3.csv.png': 0.44989267,
 'Q2_cluster_93.csv.png': 0.23741737,
 'Q3_cluster_1.csv.png': 0.14096485,
 'Q3_cluster_13.csv.png': 0.1951197,
 'Q3_cluster_23.csv.png': 0.06271134,
 'Q3_cluster_25.csv.png': 0.096590005,
 'Q3_cluster_34.csv.png

In [15]:
# Predict on the unknown images
predictions = model.predict(unknown_generator, steps=len(unknown_generator), verbose=1)

# Get the filenames of the unknown images
filenames = unknown_generator.filenames

# Output the filenames with their corresponding probabilities
for filename, prob in zip(filenames, predictions):
    print(f"{filename}: Probability of 'cluster' = {prob[0]:.4f}, Probability of 'not cluster' = {1 - prob[0]:.4f}")


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32,), but the yielded element was [[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]].
Traceback (most recent call last):

  File "c:\Users\juang\anaconda3\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 20